In [1]:
!pip install stable_baselines3
!pip install svgpath2mpl
!pip install gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616799 sha256=c08248d9b989e603d7e8440a6b63a78a3f81d61fa0103b6348cc8009244220f1
  Stored in directory: /root/.cache/pip/wheels/b3/50/6c/0a82c1358b4da2dbd9c1bb17e0f89467db32812ab236dbf6d5
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Successfully uninstalled importlib-metadata-4.12.0


In [2]:
import gym
import torch 
import torch.nn as nn
import numpy as np
from typing import Callable, Dict, List, Optional, Tuple, Type, Union
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy
from environments.shared_wildfire_gym import SharedWildFireGym

ModuleNotFoundError: No module named 'svgpath2mpl'

In [ ]:
class CustomCNN(BaseFeaturesExtractor):

    def _get_conv_out(self):
        o = self.conv(torch.zeros(1, 2, 100, 100))
        return int(np.prod(o.size()))

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 4):
        super(CustomCNN, self).__init__(observation_space, features_dim)

        self.fc1  = nn.Sequential(
            nn.Linear(5, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU()
        )

        self.conv = nn.Sequential(
            nn.Conv2d(2, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Flatten()
        )
    
        conv_out_size = self._get_conv_out()

        self.fc2 = nn.Sequential(
        nn.Linear(conv_out_size, 500),
        nn.ReLU(),
        nn.Linear(500, 100),
        nn.ReLU(),
        )

        self.flatten = nn.Sequential(nn.Linear(150, features_dim), nn.ReLU())

    def forward(self, observations):
        input = torch.cat(
            [
                observations['bank_angle'], 
                observations['rho'], observations['theta'], 
                observations['psi'], 
                observations['other_bank_angle']
            ], dim=1)
        fc1_out = self.fc1(input)
        conv_out = torch.flatten(self.conv(observations['belief_map']),1)
        fc2_out = self.fc2(conv_out)
        return self.flatten(torch.cat((fc1_out, fc2_out), dim=1))



policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256),
)


In [ ]:
wildFireGym = SharedWildFireGym()
model = PPO("MultiInputPolicy", wildFireGym, verbose=3, n_epochs=5, policy_kwargs=policy_kwargs)
model.learn(total_timesteps=2000000)
model.save("policy")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 301      |
|    ep_rew_mean     | 364      |
| time/              |          |
|    fps             | 43       |
|    iterations      | 1        |
|    time_elapsed    | 47       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 233         |
|    ep_rew_mean          | 63          |
| time/                   |             |
|    fps                  | 43          |
|    iterations           | 2           |
|    time_elapsed         | 94          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007488628 |
|    clip_fraction        | 0.0244      |
|    clip_range           | 0.2         |
|    entropy_loss  